This script is used to create WRF domain boundaries from user input.  It's supposed to help make it a little more simple to create nested domains in WRF without having to think too hard about the specific variables in the namelist.input file.

In [1]:
# import functions
# OS interaction and time
import os
import sys
import cftime
import datetime
import time
import glob
import dask
import dask.bag as db
import calendar
import importlib

# math and data
import math
import numpy as np
import netCDF4 as nc
import xarray as xr
import scipy as sp
import scipy.linalg
from scipy.signal import detrend
import pandas as pd
import pickle as pickle
from sklearn import linear_model
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing
import statsmodels.stats.multitest as multitest

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg
from matplotlib.colors import TwoSlopeNorm
import matplotlib.cm as cm

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.axes_divider import HBoxDivider
import mpl_toolkits.axes_grid1.axes_size as Size
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

# random
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# paths to various directories
rda_era5_path = '/glade/campaign/collections/rda/data/ds633.0/'  # base path to ERA5 data on derecho
my_era5_path = '/glade/u/home/zcleveland/scratch/ERA5/'  # path to subset data
misc_data_path = '/glade/u/home/zcleveland/scratch/misc_data/'  # path to misc data
plot_out_path = '/glade/u/home/zcleveland/NAM_soil-moisture/ERA5_analysis/plots/'  # path to generated plots
scripts_main_path = '/glade/u/home/zcleveland/NAM_soil-moisture/scripts_main/'  # path to my dicts, lists, and functions

# import variable lists and dictionaries
if scripts_main_path not in sys.path:
    sys.path.insert(0, scripts_main_path)  # path to file containing these lists/dicts
if 'get_var_data' in sys.modules:
    importlib.reload(sys.modules['get_var_data'])
if 'my_functions' in sys.modules:
    importlib.reload(sys.modules['my_functions'])
if 'my_dictionaries' in sys.modules:
    importlib.reload(sys.modules['my_dictionaries'])

# import common functions that I've created
from get_var_data import get_var_data, get_var_files, open_var_data, subset_var_data, time_to_year_month_avg, time_to_year_month_sum, time_to_year_month
from my_functions import month_num_to_name, ensure_var_list

# import lists and dictionaries
from my_dictionaries import (
sfc_instan_list, sfc_accumu_list, pl_var_list, derived_var_list, invar_var_list,
NAM_var_list, region_avg_list, flux_var_list, vector_var_list, misc_var_list,
var_dict, var_units, region_avg_dict, region_avg_coords, region_colors_dict
)

In [2]:
class Domain:
    def __init__(self, parent_id, parent_grid_ratio, west, east, north, south):
        """
        Initialize a Domain object.

        Parameters
        ----------
        parent_id: int
            The ID of the parent domain.
        parent_grid_ratio: int
            The grid ratio with respect to the parent domain.
        west: float
            The western boundary of the domain.
        east: float
            The eastern boundary of the domain.
        north: float
            The northern boundary of the domain.
        south: float
            The southern boundary of the domain.
        """
        self.parent_id = parent_id
        self.parent_grid_ratio = parent_grid_ratio
        self.west = west
        self.east = east
        self.north = north
        self.south = south

    def __repr__(self):
        """
        Provide a string representation of the Domain object.

        Returns
        -------
        str: A string that lists the attributes and their values.
        """
        attrs = vars(self)
        return ', '.join(f"{key}={value}" for key, value in attrs.items())

class Domains:
    def __init__(self, *all_domains):
        """
        Initialize a Domains object which holds multiple Domain objects.

        Parameters
        ----------
        *all_domains (Domain): A variable number of Domain objects.
        """
        for i, domain in enumerate(all_domains):
            setattr(self, f"d{i+1}", domain)

    def __repr__(self):
        """
        Provide a string representation of the Domains object.

        Returns
        -------
        str: A string that lists each Domain object with its identifier.
        """
        return ', '.join(f"d{i+1}={getattr(self, f'd{i+1}')}" for i in range(len(vars(self))))

    def calc_central_point(self):
        """
        Calculate the central latitude and longitude for each domain and add these
        as attributes to each domain.
        """
        for domain_name in vars(self):
            dom = getattr(self, domain_name)
            dom.center_lat = np.floor((dom.north + dom.south) / 2).astype(int)
            dom.center_lon = np.floor((dom.west + dom.east) / 2).astype(int)

    def calc_resolution(self, dx, dy):
        """
        Calculate the resolution (dx and dy) for each domain and add these
        as attributes to each domain.

        Parameters
        ----------
        dx: float
            The resolution in the x direction for the parent domain.
        dy: float
            The resolution in the y direction for the parent domain.
        """
        for domain_name in vars(self):
            dom = getattr(self, domain_name)
            if domain_name == 'd1':
                dom.dx = dx
                dom.dy = dy
            else:
                pdom = getattr(self, f"d{dom.parent_id}")  # parent domain
                dom.dx = np.floor(pdom.dx / dom.parent_grid_ratio).astype(int)
                dom.dy = np.floor(pdom.dy / dom.parent_grid_ratio).astype(int)

    def calc_num_gridpoints(self, ref_lat, ref_lon):
        """
        Calculate the number of grid points in each domain.

        Parameters
        ----------
        ref_lat: float
            The reference latitude used to determine the degree resolution of the domain in the east-west direction
        ref_lon: float
            The reference longitude. Not really used here but I didn't want it to feel left out.
        """
        for domain_name in vars(self):
            dom = getattr(self, domain_name)
            dom.e_we = np.floor((dom.east - dom.west) / (dom.dx / (111000 * np.cos(np.radians(ref_lat))))).astype(int)
            dom.e_sn = np.floor((dom.north - dom.south) / (dom.dy / 111000)).astype(int)

    def calc_ij_start(self, ref_lat, ref_lon):
        """
        Calculate the i_parent_start and j_parent_start of each domain relative to its parent domain
        """
        for domain_name in vars(self):
            dom = getattr(self, domain_name)
            if domain_name == 'd1':
                dom.i_parent_start, dom.j_parent_start = 1, 1
            else:
                pdom = getattr(self, f"d{dom.parent_id}")  # parent domain
                dom.i_parent_start = np.floor((dom.west - pdom.west) / (pdom.dx / (111000 * np.cos(np.radians(ref_lat))))).astype(int)
                dom.j_parent_start = np.floor((dom.south - pdom.south) / (pdom.dy / 111000)).astype(int)

    def normalize_longitude(self, max_lon=180):
        """
        Normalize a given set of longitudes. Converts 0-360 grids into -180 to 180 and vice versa.
    
        Parameters
        ----------
        max_lon: float or int, optional, default: 180
            The max longitude allowed. Choices are 180 for a -180 to 180 grid or 360 for a 0-360 grid
        """
        for domain_name in vars(self):
            dom = getattr(self, domain_name)
            if max_lon == 180:
                dom.west = ((dom.west + 180) % 360) - 180
                dom.east = ((dom.east + 180) % 360) - 180
            elif max_lon == 360:
                dom.west = ((dom.west + 360) % 360)
                dom.east = ((dom.east + 360) % 360)

In [3]:
def main(
    dx=111000, dy=111000, *all_domains, **kwargs
):
    # unpack domains and store into dictionary labelled domain1, domain2, ..., domainN
    domains = Domains(*all_domains)

    # make sure longitudes are -180 to 180 grid
    domains.normalize_longitude(180)

    # calculate central point of domains
    domains.calc_central_point()
    ref_lat, ref_lon = domains.d1.center_lat, domains.d1.center_lon  # reference lat/lon for dx/dy

    # calculate resolution of each domain
    domains.calc_resolution(dx, dy)
    
    # calculate number of gridpoints in each domain
    domains.calc_num_gridpoints(ref_lat, ref_lon)

    # calculate i and j parent start for each domain
    domains.calc_ij_start(ref_lat, ref_lon)
    return domains, ref_lat, ref_lon

In [8]:
if __name__ == "__main__":
    dx = 27000
    dy = 27000
    domains, ref_lat, ref_lon = main(
        dx, dy,
        Domain(
            parent_id = 1,
            parent_grid_ratio = 1,
            west = -170,
            east = -50,
            north = 65,
            south = 5,
        ),
        Domain(
            parent_id = 1,
            parent_grid_ratio = 3,
            west = -135,
            east = -65,
            north = 52,
            south = 12,
        ),
        Domain(
            parent_id = 2,
            parent_grid_ratio = 3,
            west = -120,
            east = -100,
            north = 40,
            south = 20,
        ),
        # Domain(
        #     parent_id = 2,
        #     parent_grid_ratio = 3,
        #     west = 220,
        #     east = 300,
        #     north = 50,
        #     south = 0,
        # ),
    )


In [9]:
# print params
for domain_name in vars(domains):
    dom = getattr(domains, domain_name)
    print(f"parent_id: {dom.parent_id}")
    print(f"parent_grid_ratio: {dom.parent_grid_ratio}")
    print(f"i_parent_start: {dom.i_parent_start}")
    print(f"j_parent_start: {dom.j_parent_start}")
    print(f"e_we: {dom.e_we}")
    print(f"e_sn: {dom.e_sn}")
    print(f"ref_lat: {ref_lat}")
    print(f"ref_lon: {ref_lon}\n\n")

parent_id: 1
parent_grid_ratio: 1
i_parent_start: 1
j_parent_start: 1
e_we: 404
e_sn: 246
ref_lat: 35
ref_lon: -110


parent_id: 1
parent_grid_ratio: 3
i_parent_start: 117
j_parent_start: 28
e_we: 707
e_sn: 493
ref_lat: 35
ref_lon: -110


parent_id: 2
parent_grid_ratio: 3
i_parent_start: 151
j_parent_start: 98
e_we: 606
e_sn: 740
ref_lat: 35
ref_lon: -110


